In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

!pip install statsmodels
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf  
from statsmodels.tsa.stattools import adfuller as ADF  
from statsmodels.graphics.tsaplots import plot_pacf    
from statsmodels.stats.diagnostic import acorr_ljungbox  
from statsmodels.tsa.arima_model import ARIMA

!pip install pmdarima
import pmdarima as pm
from pmdarima.arima import auto_arima

from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error,explained_variance_score,mean_absolute_error,r2_score
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
# import mobility data set

url = 'Mobility_Report.csv'
df= pd.read_csv(url)

In [ ]:
# Filter out data about London

London = df[df['sub_region_1']=='Greater London']

In [ ]:
London.info()

In [ ]:
### London['sub_region_2'].unique()

In [ ]:
SampleSet=London.drop(['retail_and_recreation_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
                       'residential_percent_change_from_baseline'],axis=1)
SampleSet.sample(10)

In [ ]:
# Covert data type

London['date'] = pd.to_datetime(London['date'],format='%d/%m/%Y')
#London['date'] = London['date'].dt.date

In [ ]:
# Select only mobility for groceries

London1 = London[['sub_region_1','sub_region_2','date','grocery_and_pharmacy_percent_change_from_baseline']]

In [ ]:
London1 = London1.pivot(index='sub_region_2', columns='date', values='grocery_and_pharmacy_percent_change_from_baseline')

In [ ]:
London1.index = pd.Series(London1.index).replace(np.nan, 'London_total')

In [ ]:
London1 = London1.T
London1[:50]

In [ ]:
#London1.to_csv('London_mobility.csv')

In [ ]:
London1[:130].plot(figsize=(30,30))

In [ ]:
# Plot time series of each borough

fig, axs = plt.subplots(7,5,figsize=(30,30))
fig.suptitle('Series')
lims = [np.datetime64('2020-02-15'), np.datetime64('2021-05-28')]
for i in range(7):
    for j in range(5):
        if i*5+j>len(London1.columns): # pass the others that we can't fill
            continue
        axs[i, j].set_title(London1.columns[i*5+j])
        axs[i, j].plot(London1[London1.columns[i*5+j]].values)
        for labels in axs[i,j].get_xticklabels():
            labels.set_rotation(40)
            labels.set_horizontalalignment('right')
            
plt.savefig('TimeSeries.png') 
plt.show()


In [ ]:
# Plot data for the study timeframe

test = London1[:130]

fig, axs = plt.subplots(7,5,figsize=(30,30))
fig.suptitle('Series')

for i in range(7):
    for j in range(5):
        if i*5+j>len(test.columns): # pass the others that we can't fill
            continue
        axs[i, j].set_title(test.columns[i*5+j])
        axs[i, j].plot(test[test.columns[i*5+j]].values)
        for labels in axs[i,j].get_xticklabels():
            labels.set_rotation(40)
            labels.set_horizontalalignment('right')
#plt.savefig('TimeSeries2.pdf',dpi=1200) 
plt.show()

In [ ]:
#Fig5_1 = London1.drop('London_total',axis=1).T
#Fig5_1.to_csv('Fig5_1.csv')

In [ ]:
Fig5_1 = pd.read_csv('Fig5_1.csv')

In [ ]:
Fig5_1.index = Fig5_1['sub_region_2']
Fig5_1 = Fig5_1.drop('sub_region_2',axis=1)

In [ ]:
# heatmap of time series

heat_map = plt.figure(figsize=(30,20))
sns_plot = sns.heatmap(Fig5_1,center=0, cmap="RdBu_r",vmin=-100,vmax=100,xticklabels=30)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Borough',fontsize=18)
sns_plot.vlines([30,310,409],0,1000,color='black',lw=2,ls='-.')
sns_plot.set_yticklabels(sns_plot.get_ymajorticklabels(), fontsize = 12)
#fig.savefig("heatmap.pdf", bbox_inches='tight') 
#plt.savefig('TimeSeriesheatmap.png') 
plt.show()

In [ ]:
Fig5_2 = Fig5_1.T
Fig5_2

In [ ]:
phase1 = plt.figure(figsize=(30,20))
sns_plot = sns.heatmap(Fig5_2[20:80].T,cmap= "RdBu_r",center=-40,xticklabels=5)
sns_plot.vlines([10,15,24],0,35,color='black',lw=2,ls='-.')
# fig.savefig("heatmap.pdf", bbox_inches='tight')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Borough',fontsize=18)
plt.savefig('Fig5_2.png') 
plt.show()

In [ ]:
# ARIMA test
ARIMA1 = London1['London_total']
ARIMA1

In [ ]:
ARIMA1[:35]

In [ ]:
# decompose time series to for in-depth exploration
#decomposition = sm.tsa.seasonal_decompose(ARIMA1[:30],model='additive')
#fig = decomposition.plot()
#fig.show()

In [ ]:
train = ARIMA1[:25]
valid = ARIMA1[25:30]
treat = ARIMA1[30:35]

In [ ]:
# auto arima test
model = auto_arima(train,start_p=1, start_q=1,
                         test='adf',m=7,
                         start_P=0, seasonal=True,
                         d=None, D=1,trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True,information_criterion='aic')
model.fit(train)
 
forecast = model.predict(n_periods=len(valid))
forecast = pd.DataFrame(forecast,index = ARIMA1[25:30].index,columns=['Prediction'])
 
#plot the predictions for validation set
plt.plot(train, label='Train')
plt.plot(valid, label='Valid')
plt.plot(treat,label='Treat')
plt.plot(forecast, label='Prediction')
plt.show()


In [ ]:
London_train = London1[:25]
London_valid = London1[25:35]

In [ ]:
# apply auto_arima to each borough

ARIMA_test = pd.DataFrame(index = London_valid.index)

for i in range(34):
    train = London_train.iloc[:,i]
    model = auto_arima(train,start_p=1, start_q=1,
                         test='adf',m=7,
                         start_P=0, seasonal=True,
                         d=None, D=1,trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True,information_criterion='aic')
    model.fit(train)
    forecast = model.predict(n_periods=10)
    #forecast = pd.DataFrame(forecast,index = London_valid.index,columns=[London_train.columns[i]])
    ARIMA_test[London_train.columns[i]] = forecast

In [ ]:
# Control group fo DiD model

After = ARIMA_test[5:]
After = After.T
After

In [ ]:
# Control group fo DiD model

Control = London1[0:30]
Control = Control.T
Control = Control.merge(After,left_index=True,right_index=True)
Control = Control.T
Control

In [ ]:
# Treat group fo DiD model

Treat = London1[:35]
Treat

In [ ]:
# Control group fo DiD model(test)

Control1 = pd.DataFrame(Control.iloc[:,0])
Control1['datee'] = Control1.index
Control1['time'] = 0
Control1['time'] = Control1['time'].where(Control1.index < '2020-03-16',1)
Control1.index = range(0,len(Control1))
Control1['Treat'] = 0
Control1

In [ ]:
# Treat group fo DiD model(test)

Treat1 = pd.DataFrame(Treat.iloc[:,0])
Treat1['datee'] = Treat1.index
Treat1['time'] = 0
Treat1['time'] = Treat1['time'].where(Treat1.index < '2020-03-16',1)
Treat1.index = range(0,len(Treat1))
Treat1['Treat'] = 1
Treat1

In [ ]:
DID = pd.concat([Control1,Treat1],axis=0)

In [ ]:
# DiD model(test)
model = 'DID.iloc[:,0] ~ Treat + time + time * Treat'

In [ ]:
mod = smf.ols(formula = model, data = DID)
res = mod.fit()
res.summary()

In [ ]:
coEff = pd.DataFrame(res.params)
coEff

In [ ]:
# DiD model to each borough 

coEff = pd.DataFrame()

for i in range(34):
    Control1 = pd.DataFrame(Control.iloc[:,i])
    Control1['datee'] = Control1.index
    Control1['time'] = 0
    Control1['time'] = Control1['time'].where(Control1.index < '2020-03-16',1)
    Control1.index = range(0,len(Control1))
    Control1['Treat'] = 0

    Treat1 = pd.DataFrame(Treat.iloc[:,i])
    Treat1['datee'] = Treat1.index
    Treat1['time'] = 0
    Treat1['time'] = Treat1['time'].where(Treat1.index < '2020-03-16',1)
    Treat1.index = range(0,len(Treat1))
    Treat1['Treat'] = 1

    DID = pd.concat([Control1,Treat1],axis=0)

    model = 'DID.iloc[:,0] ~ Treat + time + time * Treat'

    mod = smf.ols(formula = model, data = DID).fit()

    coEff[Control.columns[i]] = mod.params

coEff


In [ ]:
# coefficient matrix

coEff = coEff.T
#coEff.to_csv('DID_result.csv')
coEff

In [ ]:
# import map for visualisation

path = 'https://github.com/jreades/i2p/blob/master/data/src/Boroughs.gpkg?raw=true'
map_bor = gpd.read_file(path)
map_bor

In [ ]:
# DID result

MobilityDecrease = pd.read_csv('DID_result.csv')
MobilityDecrease

In [ ]:
MobilityDecrease['Borough'] = MobilityDecrease[MobilityDecrease.columns[0]]
MobilityDecrease = MobilityDecrease.drop([MobilityDecrease.columns[0]],axis=1)

In [ ]:
map1 = map_bor.merge(MobilityDecrease,left_on='NAME', right_on='Borough')
map1

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,20),)
map1.plot(column='time:Treat',ax=ax, legend=True,cmap='coolwarm')

#plt.savefig('map1.png')
#files.download("map1.png") 

In [ ]:
# Moran's I

LocalMoran = pd.read_csv('localMoran.csv')
maplm = map_bor.merge(LocalMoran,left_on='NAME', right_on='Borough')
maplm

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,20),)
maplm.plot(column='Stage1',ax=ax, legend=True,cmap='RdBu_r',vmin=-1,vmax=1)

In [ ]:
# grocery data set

url2 = 'London_GroceryPoints.csv'
grocerypoint = pd.read_csv(url2)
grocerypoint.rename(columns={'NAME':'Borough'}, inplace = True)
grocerypoint.sample(5)

In [ ]:
grocerypoint.info()

In [ ]:
grocerypoint.groupby('retailer').count()

In [ ]:
grocerypoint['size_band'].unique()

In [ ]:
grocerypoint.groupby('size_band').count()

In [ ]:
groceryCom = pd.DataFrame(grocerypoint.groupby(['size_band','Borough']).size())
groceryCom = groceryCom.unstack(level=0,fill_value=0)
#groceryCom = groceryCom.T

#groceryCom.index = range(0,len(groceryCom))
groceryCom

In [ ]:
groceryCom1 = groceryCom.div(groceryCom.sum(axis=1), axis=0)
groceryCom1

In [ ]:
# Journeytime data set

Journeytime = pd.read_csv('JourneyTime.csv')
Journeytime = Journeytime.drop(['LSOA_code','Region','LA_Code'],axis=1)
Journeytime = Journeytime.groupby(['LA_Name']).mean()

Journeytime

In [ ]:
# population by age group

AgeGroup = pd.read_csv('populationByAge.csv')
AgeGroup = AgeGroup.drop(['Code',' ALL AGES'],axis=1)
#Journeytime = Journeytime.groupby(['LA_Name']).mean()

AgeGroup.index = AgeGroup['Area']
AgeGroup = AgeGroup.drop(['Area'],axis=1)
AgeGroup

In [ ]:
AgeGroup1 = AgeGroup.div(AgeGroup.sum(axis=1), axis=0)
AgeGroup1['Total_pop'] = AgeGroup.sum(axis=1)
AgeGroup1

In [ ]:
# Points within a distance

NumPoints = pd.read_csv('NumPoints_Distance_byBorough.csv')
NumPoints.index = NumPoints['NAME']
NumPoints = NumPoints.drop(['GSS_CODE','NAME'],axis=1)
NumPoints

In [ ]:
# Income deprivation

IMD = pd.read_csv('IMD.csv')
IMD.index = IMD['Borough']
IMD = IMD.drop(['Borough'],axis=1)

IMD

In [ ]:
# Car ownership

CarOwnership = pd.read_csv('CarOwnership.csv')
CarOwnership.index = CarOwnership['Borough']
CarOwnership = CarOwnership.drop(['Borough'],axis=1)

CarOwnership

In [ ]:
CarOwnership1 = CarOwnership.drop(['Cars: sum of All cars or vans in the area; measures: Value'],axis=1)
CarOwnership1 = CarOwnership1.div(CarOwnership1.sum(axis=1), axis=0)
CarOwnership1['Total_car'] = CarOwnership['Cars: sum of All cars or vans in the area; measures: Value']
CarOwnership1

In [ ]:
# Distance to grocery store

DistanceToGrocery = pd.read_csv('LSOA_grocerypoints.csv')
g = DistanceToGrocery.groupby(['Borough'])
newdf = g.apply(lambda x: pd.Series([np.average(x['HubDist'], weights=x['All Ages']), 
                             np.average(x['NUMPOINTS('], weights=x['All Ages']),np.average(x['NUMPOINT_1'], weights=x['All Ages']),np.average(x['NUMPOINT_2'], weights=x['All Ages']),], 
                                    index=['WA_HubDist','WA_NUMPOINTS(500m)','WA_NUMPOINTS(1km)','WA_NUMPOINTS(2km)']))
newdf

In [ ]:
map2 = map1.merge(groceryCom1,left_on='NAME', right_on= groceryCom1.index)
map2 = map2.merge(Journeytime,left_on='NAME', right_on= Journeytime.index)
map2 = map2.merge(NumPoints,left_on='NAME', right_on= NumPoints.index)
map2 =  map2.merge(AgeGroup1,left_on='NAME', right_on= AgeGroup1.index)
map2 =  map2.merge(IMD,left_on='NAME', right_on= IMD.index)
map2 =  map2.merge(CarOwnership1,left_on='NAME', right_on= CarOwnership1.index)
map2 = map2.merge(newdf,left_on='NAME', right_on= newdf.index)

map2

In [ ]:
# population density

map2['pop_den'] = map2['Total_pop']/map2['HECTARES']
map2

In [ ]:
# Nearest Neighbour Index

NNI = pd.read_csv('NNIresult.csv')
map2 = map2.merge(NNI,left_on='NAME', right_on= 'Borough')
map2

In [ ]:
# workplace population

workplace = pd.read_csv('workplace.csv')
workplace.index = workplace['Borough']
workplace = workplace.drop(['Borough'],axis=1)
workplace

In [ ]:
map2 = map2.merge(workplace,left_on='NAME', right_on= workplace.index)
map2

In [ ]:
map2['employee/pop'] = map2['Employee']/map2['Total_pop']
map2

In [ ]:
correff = map2.drop(['GSS_CODE','HECTARES','NONLD_AREA','ONS_INNER','geometry','Intercept','Treat','time','Borough_x','Borough_y'],axis=1)
correff.index = correff['NAME']
correff = correff.drop(['NAME','FoodPT45n','FoodPT60n','FoodPT45pct','FoodPT60pct','FoodCar30n','FoodCar45n',
                        'FoodCar60n','FoodCar15pct','FoodCar30pct','FoodCar45pct','FoodCar60pct'],axis=1)
correff
#correff = correff.drop(['City of London','Tower Hamlets'],axis=0)
correff.to_csv('correff1.csv')

In [ ]:
corrStage1 = pd.read_csv('correffStage1.csv')
corrStage1

In [ ]:
# correlation analysis

corrMatrix = correff.corr()
#f, ax = plt.subplots(figsize=(50, 50))
#sns.heatmap(corrMatrix, vmax=.8, square=True,annot=True,cmap='BrBG')

#plt.savefig('heatmap.png', dpi=300)
#plt.show()
corrMatrix.info()

In [ ]:
corrMatrix
corrMatrix.to_csv('corrMatrixStage1.csv')

In [ ]:
correfftest = pd.read_csv('correff1.csv')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(correfftest.drop(['time:Treat','NAME'], axis=1), correfftest['time:Treat'], test_size = 0.10, random_state = 0)

In [ ]:
# Regression Model Test 1 
# increase the max_iter to guarantee convergence
lasso_model = linear_model.Lasso(max_iter=10e7, normalize=True)
lasso_model.fit(X=X_train, y=y_train)

In [ ]:
# Regression Model Test 2
reg_tree = DecisionTreeRegressor(random_state=0)
reg_tree.fit(X=X_train, y=y_train)

In [ ]:
reg_tree.score(X=X_train, y=y_train)

In [ ]:
# Random Forest Regression Model
reg_random_forest = RandomForestRegressor(random_state=0,oob_score=True)
reg_random_forest.fit(X_train, y_train.values.reshape(-1))

In [ ]:
#from sklearn.model_selection import GridSearchCV
#Grid_List = {'max_depth':[int(x) for x in np.linspace(10, 100, num = 10)],'n_estimators':[int(x) for x in np.linspace(start = 10, stop = 200, num = 10)]}
#grid_search = GridSearchCV(estimator=reg_random_forest,cv=5,param_grid=Grid_List)
#grid_search.fit(X=X_train, y=y_train.values.reshape(-1))

In [ ]:
#CVResults = pd.DataFrame(grid_search.cv_results_)
#CVResults

In [ ]:
reg_random_forest.score(X=X_train, y=y_train)

In [ ]:
reg_random_forest.score(X=X_test, y=y_test)

In [ ]:
y_pred = reg_random_forest.predict(X_test)
y_pred

In [ ]:
y_test

In [ ]:
mean_squared_error(y_test,y_pred)

In [ ]:
reg_random_forest.feature_importances_

In [ ]:
# grocery data set(LSOA)

urlLSOA = 'GrocerybyLSOA.csv'
grocerypointLSOA = pd.read_csv(urlLSOA)
#grocerypointLSOA.rename(columns={'NAME':'Borough'}, inplace = True)
grocerypointLSOA.info()

In [ ]:
groceryComLSOA = pd.DataFrame(grocerypointLSOA.groupby(['size_band','LSOA11CD']).size())
groceryComLSOA = groceryComLSOA.unstack(level=0,fill_value=0)
#groceryCom = groceryCom.T

#groceryCom.index = range(0,len(groceryCom))
groceryComLSOA = groceryComLSOA.div(groceryComLSOA.sum(axis=1), axis=0)
groceryComLSOA

In [ ]:
# population by age group LSOA

AgeGroupLSOA = pd.read_csv('LSOA_population.csv')
AgeGroupLSOA.index = AgeGroupLSOA['LSOA Code']
AgeGroupLSOA = AgeGroupLSOA.drop(['LSOA Code'],axis=1)
AgeGroupLSOA

In [ ]:
AgeGroupLSOA1 = AgeGroupLSOA.div(AgeGroupLSOA.sum(axis=1), axis=0)
AgeGroupLSOA1['Total_pop'] = AgeGroupLSOA.sum(axis=1)
AgeGroupLSOA1

In [ ]:
# Number of points at LSOA level

NumPointsLSOA = pd.read_csv('LSOA_NumPoints.csv')
NumPointsLSOA.index = NumPointsLSOA['LSOA11 Code']
NumPointsLSOA = NumPointsLSOA.drop(['LSOA11 Code'],axis=1)
NumPointsLSOA = NumPointsLSOA.fillna(0)
NumPointsLSOA.rename(columns={'id_count':'NumPoint','NUMPOINTS(':'NUMPOINTS(500m)','NUMPOINT_1':'NUMPOINTS(1km)','NUMPOINT_2':'NUMPOINTS(2km)'}, inplace = True)
NumPointsLSOA['Area'] = 10*NumPointsLSOA['Area']
NumPointsLSOA

In [ ]:
# Car ownership LSOA level

CarOwnershipLSOA = pd.read_csv('LSOA_carownership1.csv')
CarOwnershipLSOA.index = CarOwnershipLSOA['lsoa11cd']
CarOwnershipLSOA = CarOwnershipLSOA.drop(['lsoa11cd'],axis=1)

CarOwnershipLSOA

In [ ]:
CarOwnershipLSOA1 = CarOwnershipLSOA.drop(['Cars: sum of All cars or vans in the area; measures: Value'],axis=1)
CarOwnershipLSOA1 = CarOwnershipLSOA1.div(CarOwnershipLSOA1.sum(axis=1), axis=0)
CarOwnershipLSOA1['Total_car'] = CarOwnershipLSOA['Cars: sum of All cars or vans in the area; measures: Value']
CarOwnershipLSOA1

In [ ]:
# Journey time LSOA level

LSOAJourneytime = pd.read_csv('LSOA_JourneyTime.csv')
LSOAJourneytime.index = LSOAJourneytime['LSOA_code']
LSOAJourneytime = LSOAJourneytime.drop(['LSOA_code'],axis=1)
LSOAJourneytime

In [ ]:
# Workplace population LSOA level

workplaceLSOA = pd.read_csv('LSOA_Employee1.csv')
workplaceLSOA.index = workplaceLSOA['LSOA11CD']
workplaceLSOA = workplaceLSOA.drop(['LSOA11CD'],axis=1)
workplaceLSOA

In [ ]:
path1 = 'LSOA_London.gpkg'
map_lsoa = gpd.read_file(path1)
map_lsoa

In [ ]:
LSOA_variables = pd.DataFrame(map_lsoa['LSOA11CD'])
LSOA_variables = LSOA_variables.merge(groceryComLSOA,right_on=groceryComLSOA.index, left_on='LSOA11CD',how='left').fillna(0)
LSOA_variables = LSOA_variables.merge(LSOAJourneytime,right_on=LSOAJourneytime.index, left_on='LSOA11CD')
LSOA_variables = LSOA_variables.merge(NumPointsLSOA,right_on=NumPointsLSOA.index, left_on='LSOA11CD')
LSOA_variables = LSOA_variables.merge(AgeGroupLSOA1,right_on=AgeGroupLSOA1.index, left_on='LSOA11CD')
LSOA_variables = LSOA_variables.merge(CarOwnershipLSOA1,right_on=CarOwnershipLSOA1.index, left_on='LSOA11CD')
LSOA_variables = LSOA_variables.merge(workplaceLSOA,right_on=workplaceLSOA.index, left_on='LSOA11CD')

LSOA_variables['pop_den'] = LSOA_variables['Total_pop']/LSOA_variables['Area']
LSOA_variables['employee/pop'] = LSOA_variables['Employee']/LSOA_variables['Total_pop']
LSOA_variables.index = LSOA_variables['LSOA11CD']
LSOA_variables = LSOA_variables.drop(['LSOA11CD'],axis=1)

#LSOA_variables.to_csv('LSOA_variables.csv')

LSOA_variables

In [ ]:
LSOA_variables1 = pd.read_csv('LSOA_variables_p.csv')
LSOA_variables1.index = LSOA_variables1['LSOA11CD']
LSOA_variables1 = LSOA_variables1.drop(['LSOA11CD'],axis=1)
LSOA_variables1

In [ ]:
# Prediction

lsoa_pred = pd.DataFrame(reg_random_forest.predict(LSOA_variables1))
lsoa_pred.index = LSOA_variables1.index
#soa_pred.to_csv('Stage1pre.csv')

lsoa_pred

In [ ]:
Stage1 = pd.read_csv('Stage1pre.csv')
Stage1.index = Stage1['LSOA11CD']
Stage1 = Stage1.drop(['LSOA11CD'],axis=1)
Stage1

In [ ]:
Stage2 = pd.read_csv('Stage2pre.csv')
Stage2.index = Stage2['LSOA11CD']
Stage2 = Stage2.drop(['LSOA11CD'],axis=1)
Stage2

In [ ]:
Stage3 = pd.read_csv('Stage3pre.csv')
Stage3.index = Stage3['LSOA11CD']
Stage3 = Stage3.drop(['LSOA11CD'],axis=1)
Stage3

In [ ]:
Offline = Stage1.merge(Stage2,left_on=Stage1.index, right_on=Stage2.index)
Offline = Offline.merge(Stage3,left_on='key_0', right_on=Stage3.index)
Offline.index = Offline['key_0']
Offline = Offline.drop(['key_0'],axis=1)
Offline
Offline.to_csv('PreLSOA.csv')

In [ ]:
# data standardization
rs = RobustScaler(quantile_range=(10.0, 90.0))

for c in Offline.columns.values:
    Offline[c] = rs.fit_transform(Offline[c].values.reshape(-1,1))
    
Offline

In [ ]:
# one-hot encoding

Online = pd.read_csv('iuc.csv')
Online.index = Online['LSOA11_CD']
Online = Online.drop(['LSOA11_CD'],axis=1)
Online1 = pd.get_dummies(Online['OnlineGroup'],prefix='OnlineGroup')
Online

In [ ]:
Indicators = Offline.merge(Online1,left_on= Offline.index, right_on=Online.index,how='left')
Indicators.index = Indicators['key_0']
Indicators = Indicators.drop(['key_0'],axis=1)

Indicators

In [ ]:
# K-means Clustering

x = []
y = []

for k in range(2,41):
    
    #############
    # Do the clustering using the main columns
    kmeans = KMeans(n_clusters=k, n_init=25, random_state=42).fit(Indicators)
    
    # Calculate the overall silhouette score
    silhouette_avg = silhouette_score(Indicators, kmeans.labels_)
    
    y.append(k)
    x.append(silhouette_avg)
    
    print('', end='')

print()
print(f"Largest silhouette score was {max(x):6.4f} for k={y[x.index(max(x))]}")
plt.figure(figsize=(18,18))
plt.plot(y, x)
plt.ylabel('Average Silhouette Scores',fontsize=18)
plt.xlabel('Number of clusters',fontsize=18)
plt.savefig('Fig5N.png') 
#plt.gcf().suptitle("Average Silhouette Scores");

In [ ]:
# calculate SSE for a range of number of cluster
list_SSE = []
min_k = 1
max_k = 10
range_k = range(min_k, max_k+1)
for i in range_k:
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(Indicators)
    # inertia is a concept in physics. Roughly it means SSE of clustering.
    list_SSE.append(km.inertia_)

# plot
plt.figure(figsize=(15,15))
plt.plot(range_k, list_SSE, marker='o')
plt.ylabel('SSE',fontsize=18)
plt.xlabel('Number of clusters',fontsize=18)
plt.savefig('Fig5N1.png') 

plt.show()

In [ ]:
k_pref=7
    
#############
# Do the clustering using the main columns
kmeans = KMeans(n_clusters=k_pref, n_init=25, random_state=42).fit(Indicators)

# Convert to a series
s = pd.Series(kmeans.labels_, index=Indicators.index, name='K-means')

# We do this for plotting
Indicators1 = Indicators
Indicators1['K-means'] = s
    
# Calculate the overall silhouette score
silhouette_avg = silhouette_score(Indicators, kmeans.labels_)

# Calculate the silhouette values
sample_silhouette_values = silhouette_samples(Indicators, kmeans.labels_)
    
#############
# Create a subplot with 1 row and 2 columns
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_size_inches(9, 5)

# The 1st subplot is the silhouette plot
# The silhouette coefficient can range from -1, 1
ax1.set_xlim([-1.0, 1.0]) # Changed from -0.1, 1
    
# The (n_clusters+1)*10 is for inserting blank space between silhouette
# plots of individual clusters, to demarcate them clearly.
ax1.set_ylim([0, Indicators1.shape[0] + (k + 1) * 10])
    
y_lower = 10
    
# For each of the clusters...
for i in range(k_pref):
    # Aggregate the silhouette scores for samples belonging to
    # cluster i, and sort them
    ith_cluster_silhouette_values = \
        sample_silhouette_values[kmeans.labels_ == i]

    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i
        
    # Set the color ramp
    color = plt.cm.Spectral(i/k)
    ax1.fill_betweenx(np.arange(y_lower, y_upper),
                        0, ith_cluster_silhouette_values,
                        facecolor=color, edgecolor=color, alpha=0.7)

    # Label the silhouette plots with their cluster numbers at the middle
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--", linewidth=0.5)

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks(np.arange(-1.0, 1.1, 0.2)) # Was: [-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1]

    # 2nd Plot showing the actual clusters formed --
    # we can only do this for the first two dimensions
    # so we may not see fully what is causing the 
    # resulting assignment
    colors = plt.cm.Spectral(kmeans.labels_.astype(float) / k)
    ax2.scatter(Indicators1[Indicators1.columns[0]], Indicators1[Indicators1.columns[1]], 
                marker='.', s=30, lw=0, alpha=0.7, c=colors)

    # Labeling the clusters
    centers = kmeans.cluster_centers_
    
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1],
                marker='o', c="white", alpha=1, s=200)

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50)

    ax2.set_title("Visualization of the clustered data")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

plt.suptitle(("Silhouette results for KMeans clustering "
                "with %d clusters" % k_pref),
                fontsize=14, fontweight='bold')

plt.show()

In [ ]:
Indicators['K-means']=kmeans.labels_
#Indicators.to_csv('KmeansResult.csv')

In [ ]:
centroids = None
for k in sorted(Indicators1['K-means'].unique()):
    print(f"Processing cluster {k}")

    c = Indicators1[Indicators1['K-means']==k]
    if centroids is None:
        centroids = pd.DataFrame(columns=c.columns.values)
    centroids = centroids.append(c.mean(), ignore_index=True)
    
odf = pd.DataFrame(columns=['Variable','Cluster','Std. Value'])
for i in range(0,len(centroids.index)):
    row = centroids.iloc[i,:]
    c_index = list(centroids.columns.values).index('K-means')
    for c in range(0,c_index):
        d = {'Variable':centroids.columns[c], 'Cluster':row[c_index], 'Std. Value':row[c]}
        odf = odf.append(d, ignore_index=True)
        
odf = odf[~odf.Variable.isin(['Borough','msoa11hclnm','Subregion','geometry'])]
g = sns.FacetGrid(odf, col="Variable", col_wrap=3, height=3, aspect=1.5, margin_titles=True, sharey=True)
g = g.map(plt.plot, "Cluster", "Std. Value", marker=".")

In [ ]:
Summary = Indicators.merge(Online,left_on= Indicators.index, right_on=Online.index,how='left')
Summary

In [ ]:
#Summary.to_csv('KmeansResult2.csv')

In [ ]:
SVL = pd.read_csv('LastDD.csv')
SVL.index = SVL['LSOA']
SVL = SVL.drop('LSOA',axis=1)
SVL

In [ ]:
c = SVL.groupby('K-means').mean()
#c.to_csv('clusterResultLL.csv')

In [ ]:
c1 = SVL.groupby('K-means')['OnlineGroup'].nunique()
c1

In [ ]:
c2 = SVL.groupby('K-means').count()
c2

In [ ]:
fig, axs = plt.subplots(7,1,figsize=(18,18))
for k in range(7):


    anne = SVL[SVL['K-means']==k]
    anne = anne['Stage3'].tolist()
    axs[k].hist(anne)
    n = k+1
    axs[k].set_ylabel('Cluster'" %d" % n)
plt.savefig('Fig5.18.png') 
plt.show()